In [1]:
 #!pip install -r requirements.txt

In [2]:
import os
import pandas as pd
import numpy as np
from dwave.system import LeapHybridCQMSampler, LeapHybridSampler
from dimod import ConstrainedQuadraticModel,  QuadraticModel, quicksum, Integer, BinaryQuadraticModel, Binary
from dotenv import load_dotenv, find_dotenv
import networkx as nx
import re
from pprint import pprint
import random
import matplotlib
try:
    import matplotlib.pyplot as plt
except ImportError:
    matplotlib.use("agg")
    import matplotlib.pyplot as plt

In [3]:
# Load Environmental Variables
load_dotenv(find_dotenv())
token = os.environ['DWAVE_API_KEY'] 

In [49]:
# Define the number of facilities/locations
n = 4

# Define the flow matric
f = np.array([[0, 3, 0, 2], [3, 0, 0, 1], [0, 0, 0, 4], [2, 1, 4, 0]])

# Define the distance matrix
d = np.array([[0, 22, 53, 53], [22, 0, 40, 62], [53, 40, 0, 55], [53, 62, 55, 0]])

offset = 0.5

In [41]:
def build_cqm():

    # Define the decision variables: Using a dictionary comprehension where each variable is a binary variable corresponding to the assignment of a facility to a location.
    x = {(i, j): Binary(f'F{i}_L{j}') for i in range(n) for j in range(n)}

    # Instantiate the Constrained Quadratic Model
    cqm = ConstrainedQuadraticModel()

    # Define the objective function: The cost is computed as the product of the flow between the facilities and the distance between the locations, multiplied by the binary variables corresponding to the assignment of each facility to each location
    objective =(offset) * quicksum(f[i][j] * d[p][q] * x[(i, p)] * x[(j, q)] for i in range(n) for j in range(n) for p in range(n) for q in range(n))

    cqm.set_objective(objective)

    # Define the constraints to ensure that each facility is assigned to exactly one location
    for i in range(n):
        cqm.add_constraint(
        quicksum(x[(i, j)] for j in range(n)) == 1
        )

    # Define the constraints to ensure that each location is assigned to exactly one facility
    for j in range(n):
        cqm.add_constraint(
        quicksum(x[(i, j)] for i in range(n)) == 1
        )
    return cqm

In [25]:
def parse_solution(cqm):

    # Solve the problem
    sampler = LeapHybridCQMSampler(token=token)
    print("Submitting CQM to solver {}.".format(sampler.solver.name))

    # Sample from the model
    sampleset = sampler.sample_cqm(cqm, label='QAP')
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
    if not len(feasible_sampleset):
        raise ValueError("No feasible solution found")
    
    # Get the lowest feasible energy solution and store the solution variable
    best = feasible_sampleset.first
    result = [(key, val) for key, val in best.sample.items() if val==1.0]

    locations_ordered = sorted(result, key=lambda x: int(x[0].split("_")[1][1:]))
    result = [int(t[0].split('_')[0][1:]) for t in locations_ordered]

    return result, best.energy


In [50]:
cqm = build_cqm()

In [51]:
facility_sequence, total_cost  = parse_solution(cqm)

Submitting CQM to solver hybrid_constrained_quadratic_model_version1.


In [48]:
print(f'Facility_sequence according to the order of location[0-indexed]: {facility_sequence}')

Facility_sequence according to the order of location[0-indexed]: [2, 0, 8, 7, 6, 3, 4, 5, 1]


In [52]:
print(f'Total Cost: {total_cost}')

Total Cost: 395.0


** Let's compare the case for n=9

In [42]:
# Define the number of facilities/locations
n = 9

# Define the flow matrix
f = np.array([[0, 2, 4, 0, 0, 0, 2, 0, 0], [2, 0, 3, 1, 0, 6, 0, 0, 2], [4, 3, 0, 0, 0, 3, 0, 0, 0],
              [0, 1, 0, 0, 1, 0, 1, 2, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 6, 3, 0, 0, 0, 0, 0, 2],
              [2, 0, 0, 1, 0, 0, 0, 4, 3], [0, 0, 0, 2, 0, 0, 4, 0, 0], [0, 2, 0, 0, 0, 2, 3, 0, 0]])

# Define the distance matrix
d = np.array([[0, 32, 68, 97, 75, 70, 75, 40, 24], [32, 0, 42, 80, 53, 65, 82, 47, 29], [68, 42, 0, 45, 15, 49, 79, 55, 50],
              [97, 80, 45, 0, 30, 36, 65, 65, 73], [75, 53, 15, 30, 0, 38, 69, 53, 53], [70, 65, 49, 36, 38, 0, 31, 32, 46],
              [75, 82, 79, 65, 69, 31, 0, 36, 56], [40, 47, 55, 65, 53, 32, 36, 0, 19], [24, 29, 50, 73, 53, 46, 56, 19, 0]])


In [43]:
cqm = build_cqm()
facility_sequence, total_cost  = parse_solution(cqm)

Submitting CQM to solver hybrid_constrained_quadratic_model_version1.


In [46]:
print(f'Facility_sequence according to the order of location[0-indexed]: {facility_sequence}')

Facility_sequence according to the order of location[0-indexed]: [2, 0, 8, 7, 6, 3, 4, 5, 1]


In [45]:
print(f'Total Cost: {total_cost}')

Total Cost: 1160.0
